Genetic Algorithm
=======

# Theory

## Definition


## Examples


# Implementation


## Imports

In [1]:
import matplotlib
import math
import numpy as np
import pandas as pd
from typing import Callable
import matplotlib.pyplot as plt
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Parameters

## Algorithm
```mermaid
    flowchart TB

        x_0(Start with starting \nsolution x_0) --> Loop
        x_0 --> population(Initial population)
        population --> Loop
        
        subgraph Loop
            direction LR

            offsprings(Selection of population \n offsprings) --> condition{Is the best offspring cost better \nthan current cost?}
            condition --> |True| move(Move to offspring)
            condition --> |False| stay(Stay where you are)
            
            move --> Mutation
            stay --> Mutation

            subgraph Mutation
                direction TB
            
                children(Produce children \nfrom offsprings using \n crossover method) --> add(Create new  population set \nfrom offsprings children)
            end

            Mutation --> update(Update population)

        end

        Loop --> return(Return current solution)

        
```

In [4]:
def GA(fitness_func: Callable, generate_children: Callable, lam: int, mu: int, x_0: float, mean: float = 0, SD: float = 1.5, generations: int = int(5e2)) -> tuple[float, list, int]:
    '''
    _summary_

    Args:
        cost_func: _description_
        get_neighborhood: _description_
        lam: _description_
        mu: _description_
        x_0: _description_
        generations: _description_. Defaults to int(5e2).

    Returns:
        _description_
    '''
    
    # algorithm data 
    population = generate_children(x_0, size=lam)
    number_children = int(lam/mu)
    cost = [fitness_func(x_0)]
    accepted_samples = 0
    x = [x_0]


    generations_population = [population]
    for _ in range(generations-1):
        
        # offsprings selection
        population_costs = np.array([fitness_func(neighbor) for neighbor in population])
        offsprings = population[ population_costs.argsort()[:mu] ]


        # check the merit of best offspring 
        best_offspring = offsprings[0]

        ## accept it if better cost
        if fitness_func(best_offspring) < fitness_func(x[-1]):
            x.append(best_offspring)
            cost.append(fitness_func(best_offspring))
            generations_population.append(population)
            accepted_samples+=1

        ## reject it "keep current best"
        else:
            x.append(x[-1])
            cost.append(fitness_func(x[-1]))
            generations_population.append(population)


        # produce new population
        offsprings_children = []
        for offspring in offsprings:
            children = generate_children(offspring, size=number_children)
            offsprings_children.extend(children)
       
        population = np.array(offsprings_children)

    x_optimal, history = x[-1], pd.DataFrame({"cost": cost, "x": x, "population": generations_population})


    return x_optimal, accepted_samples, history

## Results Visualization & Behaver Analysis


In [ ]:
def cost_function(queens: np.array) -> float:
    '''
    The cost is calculated by counting the total number of attacks on the board. The attacks considered in this cost function are vertical, 
    horizontal and diagonal attacks. It should be noted that comitative attacks are counted once "For example, a vertical attack from Queen_i 
    on Queen_j is that same as the vertical attack from Queen_j on Queen_i, hence they are counted as 1 attack together"
    '''
    # collect queens positions in vertical and right/left diagonal lines "position described by line index"
    vertical_positions = queens
    right_diagonal_positions = -1*( 1*queens - np.arange(len(queens))*-1 )
    left_diagonal_positions  = -1*( -1*queens - np.arange(len(queens))*-1 )
    
    # check for attacks, by checking if they share same line index
    cost = 0
    for queen_positions in [vertical_positions, right_diagonal_positions, left_diagonal_positions]:
        _, counts = np.unique(queen_positions, return_counts=True)
        if any(counts > 1):
           cost += sum([math.comb(queens_same_axis, 2) for queens_same_axis in counts[counts > 1]])
    
    return cost


In [ ]:
x_optimal, accepted_samples, history = GA(cost_func=cost_func,
                                          sample_neighbor=get_neighbor,
                                          x_0=x_0, 
                                          cooling_schedule=(cooling_func, cooling_window), 
                                          )

(-0.6709468886932317, 37)